<a href="https://colab.research.google.com/github/Flychuban/Malaria-Prediction-/blob/main/MalariaVGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import glob
from tensorflow.keras.layers import Lambda, Dense, Flatten, Input
from tensorflow.keras.models import Sequential, Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
import keras.utils as image
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as pyplot
import tensorflow as tf

In [ ]:
image_size = [224, 224]
train_path = '/content/drive/MyDrive/MalariaPrediction/data_images/Train'
test_path = '/content/drive/MyDrive/MalariaPrediction/data_images/Test'

In [ ]:
vgg = VGG19(input_shape=image_size + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
from keras.layers.serialization import activation
prediction = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.inputs, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 batch_size=16,
                                                 target_size=(224, 224),
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 4008 images belonging to 2 classes.


In [ ]:
test_set = train_datagen.flow_from_directory(test_path,
                                                 batch_size=16,
                                                 target_size=(224, 224),
                                                 class_mode='categorical')

Found 1008 images belonging to 2 classes.


In [ ]:
history = model.fit(training_set,
                    validation_data=test_set,
                    epochs=20,
                    steps_per_epoch = len(training_set),
                    validation_steps=len(test_set))

Epoch 1/20
251/251 [==============================] - 81s 310ms/step - loss: 0.9779 - accuracy: 0.6617 - val_loss: 0.8767 - val_accuracy: 0.6012
Epoch 2/20
251/251 [==============================] - 80s 317ms/step - loss: 0.7232 - accuracy: 0.6784 - val_loss: 0.7012 - val_accuracy: 0.6825
Epoch 3/20
251/251 [==============================] - 81s 322ms/step - loss: 0.8019 - accuracy: 0.6809 - val_loss: 1.9574 - val_accuracy: 0.5308
Epoch 4/20
251/251 [==============================] - 73s 293ms/step - loss: 0.8694 - accuracy: 0.6819 - val_loss: 0.9574 - val_accuracy: 0.6062
Epoch 5/20
251/251 [==============================] - 73s 293ms/step - loss: 0.7259 - accuracy: 0.6936 - val_loss: 0.8730 - val_accuracy: 0.6250
Epoch 6/20
110/251 [============>.................] - ETA: 35s - loss: 0.8200 - accuracy: 0.6636

In [ ]:
model.save("malaria_model3.h5")